In [1]:
from transformers import AutoTokenizer

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

from pathlib import Path
import random
import json
import shutil
import sys

import argparse

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)

/tmp/ipykernel_135773/3221944581.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
GENERATE_METHODS_DIR = Path('data/docs/manual')
METHODS_DIR = Path('data/docs/methods')
PROMPT_SEEDS_DIR = Path('data/prompts/generation/components')
PROMPT_COMPONENTS_DIR = Path('data/prompts/generation/components')
VAL_PROMPT_COMPONENTS_DIR = Path('data/prompts/validation/components')
GEN_PROMPTS_DIR = Path('data/prompts/generation/output')
VAL_PROMPTS_DIR = Path('data/prompts/validation/output')
PERSIST_DIR = Path("data/persist_dir")
OUTPUT_DIR = Path("output/")
MODELS_PATH = Path('models/')
DATA_DIR = Path('data/')

In [3]:
GT_PATH = DATA_DIR / 'datasets/merged/test_0.csv'
RUN_NAME = 'gemma_2b_it_hf_ft'
NUM_EXAMPLES = 2
NUM_NODES = 3
MODEL_NAME = 'google/gemma-1.1-2b-it'
PEFT_MODEL = "models/gemma/checkpoint-500"
N_CTX = 4000
settings = {
    'llm': MODEL_NAME,
    'num_examples': NUM_EXAMPLES,
    'num_nodes': NUM_NODES,
    'n_ctx': N_CTX,
    'peft_model': PEFT_MODEL
}

# # # # # # # # # # # # 

peft_config = PeftConfig.from_pretrained(PEFT_MODEL)

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
llm = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="cuda",
    torch_dtype=torch.float16,
    quantization_config=nf4_config
)

llm = PeftModel.from_pretrained(llm, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from local_llm_function_calling import Constrainer, JsonSchemaConstraint, Generator
from local_llm_function_calling.model.huggingface import HuggingfaceModel


schema = {
    "type": "object",
    "enforceOrder": ["name", "age"]
}

constraint = JsonSchemaConstraint(schema)
constrainer = Constrainer(HuggingfaceModel(llm))
raw_json = constrainer.generate("Generate a JSON of form: {'0': '0'}.\n", constraint, max_len=100)
truncated_json = raw_json[:constraint.validate(raw_json).end_index]

KeyboardInterrupt: 

In [6]:
truncated_json

'{\n    "name": "John Doe",\n    "age": 30\n}'